### Temp - Will be convereted into main.py once complete
Currently being used for testing

In [1]:
import torch
import pickle
import pandas as pd
import os

from data.process_data import load_data, augment_data, preprocess_data, to_tensors, split_batch
from models.quickdraw_cnn import QuickDrawCNN_V1, QuickDrawCNN_V2
from training.model_train import train

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
image_dir = "data/numpy_bitmap/"

Device: cuda


In [3]:
categories = os.listdir(image_dir)
p1, p2 = categories[:len(categories)//2], categories[len(categories)//2:]

In [4]:
bmap_p1 = load_data(p1, image_dir, file_standardize=False)
bmap_p2 = load_data(p2, image_dir, file_standardize=False)

Loading 172 categories...


Loading data: 100%|██████████| 172/172 [07:33<00:00,  2.64s/it]


Data loaded, size: (25234905, 785)
Loading 173 categories...


Loading data: 100%|██████████| 173/173 [07:37<00:00,  2.65s/it]


Data loaded, size: (25191361, 785)


In [ ]:
# Preprocessing data split array -> 28x28 numpy
bmap_p1 = preprocess_data(bmap_p1)
bmap_p2 = preprocess_data(bmap_p2)

Preprocessing 25234905 images...


  0%|          | 94511/25234905 [00:00<00:26, 937650.92it/s]

100%|██████████| 25234905/25234905 [00:12<00:00, 2088320.98it/s]
0it [05:24, ?it/s]


Preprocessing 25191361 images...


100%|██████████| 25191361/25191361 [00:13<00:00, 1828331.72it/s]
0it [01:59, ?it/s]

In [ ]:
features, labels, labels_map = to_tensors(bmap_p1.copy())
features, labels, labels_map = to_tensors(bmap_p2.copy())
bmap_p1 = list(zip(features, labels))
bmap_p2 = list(zip(features, labels))

In [ ]:
bmap_p1_1, bmap_p1_2 = bmap_p1[:len(bmap_p1) // 2], bmap_p1[len(bmap_p1) // 2:]
bamp_p2_1, bamp_p2_2 = bmap_p2[:len(bmap_p2) // 2], bmap_p1[len(bmap_p2) // 2:]

bmap_xl_p1_1 = augment_data(bmap_p1, rot=True, angle=15, h_flip=False)
bmap_xl_p1_2 = augment_data(bmap_p1, rot=True, angle=15, h_flip=False)
bmap_xl_p2_1 = augment_data(bmap_p2, rot=True, angle=15, h_flip=False)

In [ ]:
train_loader, test_loader, val_loader = split_batch(bmap, batch_size=64)

In [ ]:
model = QuickDrawCNN_V2().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss, val_loss, train_acc, val_acc = train(model = model,
                                                  train_loader = train_loader,
                                                  val_loader = val_loader,
                                                  epochs = 10,
                                                  criterion = criterion,
                                                  optimizer = optimizer,
                                                  device = device)   

In [ ]:
save_path = "models/quickdraw_cnn_v2.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}.")

In [ ]:
with open("models/labels_map.pkl", "wb") as f:
    pickle.dump(labels_map, f)
    pickle.dump(train_loss, f)
    pickle.dump(val_loss, f)
    pickle.dump(train_acc, f)
    pickle.dump(val_acc, f)